In [ ]:
import torch
import pickle
import torch.nn as nn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.optim import Adam
from simulators import *
from matrix_utils import *
from QCNN import *

In [ ]:
def get_random_computational_ids(n_qubits, n_states=32):
    return np.random.randint(2 ** n_qubits, size=(n_states))


def convert_id_to_dm(ids, n_qubits):
    N = len(ids)
    dim = 2 ** n_qubits
    out = np.zeros((N, dim, dim), dtype=np.complex64)
    for i, id_i in enumerate(ids):
        out[i, id_i, id_i] = 1.
    out_tensor = torch.tensor(out, dtype=torch.complex64)
    out_tensor = out_tensor.reshape([N] + [2] * (2 * n_qubits))
    return out_tensor


def convert_id_to_state(ids, n_qubits):
    out = np.zeros((len(ids), 1, 2**n_qubits))
    for i, id_i in enumerate(ids):
        out[i,0,id_i] = 1.
    return torch.tensor(out.reshape([len(ids),1] + [2]*(n_qubits))).type(torch.complex64)


def accuracy(x, y):
    return torch.mean((torch.sgn(x-0.5)==torch.sgn(y-0.5)).type(torch.float32))

In [ ]:
def power_schedule(n, max_mu=0.35, t_stop = 0.75):
    def f(t):
        return max(max_mu * (1 - (t / t_stop) ** n), 0)
    return f

def tanh_schedule():
    return lambda t: round(0.25*(1-np.tanh(20*(t-0.5))) / 2, 3)

def exp_schedule(max_mu, strength):
    return lambda t: round(max_mu*np.exp(-strength*t), 3)

In [ ]:
import time
def run_experiment(num_qubits,
                   schedule,
                   train_size=400,
                   batch_size=200,
                   epochs=1000,
                   lr=0.005,
                   num_samples=25,
                   verbose=5,
                   save_results=True):
    n = num_qubits
    n_batches = train_size // batch_size
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    np.random.seed(1)
    torch.manual_seed(1)
    accuracies = {'noise': [],
                  'no_noise': []}
    losses = {'noise': [],
              'no_noise': []}
    start = time.time()
    for n_exp in range(num_samples):
        # 1. Generating training dataset
        target_QCNN = QCNN(n).to(device)
        data_ids = get_random_computational_ids(n, train_size)
        y = torch.zeros(len(data_ids)).to(device)
        for batch_i in range(n_batches):
            data = convert_id_to_state(data_ids[batch_i*batch_size:(batch_i+1)*batch_size], n).to(device)
            with torch.no_grad():
                state = state_mixture_simulator(n, state_init = data, device = device)
                y[batch_i * batch_size:(batch_i + 1) * batch_size] = target_QCNN(state, mu=0.)
        # 2. Training classical model without noise
        student_QCNN = QCNN(n).to(device)
        init_nn_state = student_QCNN.state_dict()
        criterion = nn.MSELoss()
        optim = Adam(student_QCNN.parameters(), lr=lr)
        for i in range(1, epochs + 1):
            epoch_loss = 0.
            epoch_acc = 0.
            for batch_i in range(n_batches):
                optim.zero_grad()
                data = convert_id_to_state(data_ids[batch_i * batch_size:(batch_i + 1) * batch_size], n).to(device)
                state = state_mixture_simulator(n, state_init=data, device=device)
                out = student_QCNN(state, mu=0.)
                loss = criterion(out, y[batch_i * batch_size:(batch_i + 1) * batch_size])
                loss.backward()
                optim.step()

                acc = accuracy(out, y[batch_i * batch_size:(batch_i + 1) * batch_size])
                epoch_loss += loss
                epoch_acc += acc

            epoch_loss = epoch_loss / n_batches
            epoch_acc = epoch_acc / n_batches

        accuracies['no_noise'].append(epoch_acc.item())
        losses['no_noise'].append(epoch_loss.item())
        
        # 3. Training noise-induced model
        student_QCNN = QCNN(n).to(device)
        ## Starting optimization routine from the same parameters
        student_QCNN.load_state_dict(init_nn_state)
        
        student_QCNN.noise_on = True

        criterion = nn.MSELoss()
        optim = Adam(student_QCNN.parameters(), lr=lr)
        for i in range(1, epochs + 1):
            epoch_loss = 0.
            epoch_acc = 0.
            mu = schedule(i / epochs)
            for batch_i in range(n_batches):
                optim.zero_grad()
                data = convert_id_to_state(data_ids[batch_i * batch_size:(batch_i + 1) * batch_size], n).to(device)
                state = state_mixture_simulator(n, state_init=data, device=device)
                out = student_QCNN(state, mu=mu)
                loss = criterion(out, y[batch_i * batch_size:(batch_i + 1) * batch_size])
                loss.backward()
                optim.step()

                acc = accuracy(out, y[batch_i * batch_size:(batch_i + 1) * batch_size])
                epoch_loss += loss
                epoch_acc += acc

            epoch_loss = epoch_loss / n_batches
            epoch_acc = epoch_acc / n_batches

        accuracies['noise'].append(epoch_acc.item())
        losses['noise'].append(epoch_loss.item())
        if n_exp % verbose == 0:
            print(f'Sample {n_exp+1}/{num_samples}')
            print(f"Classic opt: loss is {losses['no_noise'][-1]}, accuracy is {accuracies['no_noise'][-1]}")
            print(f"Noisy opt: loss is {losses['noise'][-1]}, accuracy is {accuracies['noise'][-1]}")
            print(f'Time passed: {(time.time() - start) / 60.}m')
        if save_results:
            with open(f'opt_results/accuracies_{num_qubits}.pkl', 'wb') as f:
                pickle.dump(accuracies, f, protocol=pickle.HIGHEST_PROTOCOL)
            with open(f'opt_results/losses_{num_qubits}.pkl', 'wb') as f:
                pickle.dump(losses, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data = {'Accuracy': [], 'Number of qubits': [], 'Mode': []}
for n in [4, 6, 8, 10]:
    with open(f'opt_results/accuracies_{n}.pkl', 'rb') as f:
        acc = pickle.load(f)
        l = len(acc['noise'])
    data['Accuracy'] += acc['noise']
    data['Accuracy'] += acc['no_noise']
    data['Number of qubits'] += [n] * 2 * l
    data['Mode'] += ['Noise'] * l
    data['Mode'] += ['No noise'] * l
data = pd.DataFrame(data)

plt.figure(dpi=300)

sns.boxplot(data=data,
            orient='h',
            x='Accuracy',
            hue='Mode',
            y='Number of qubits',
            whiskerprops={'visible': True},
            medianprops={'visible': True},
            boxprops={'alpha': 1},
            palette='Set1',
            dodge=0.5,
            showfliers=False,
            showcaps=True,
            showbox=True,
            saturation=0.82)

sp = sns.swarmplot(data=data,
                   x='Accuracy',
                   hue='Mode',
                   y='Number of qubits',
                   orient='h',
                   size=4.3,
                   dodge=0.5,
                   edgecolor='k',
                   linewidth=0.5,
                   palette='Pastel1')

for collection in sp.collections:
    collection.set_label('_nolegend_')

aggregated = data.groupby(['Number of qubits', 'Mode'])['Accuracy'].max().reset_index()
pp = sns.pointplot(data=aggregated,
                   x='Accuracy',
                   y='Number of qubits',
                   hue='Mode',
                   markers='*',
                   join=False,
                   ci=None,
                   orient='h',
                   dodge=-0.395,
                   palette={'Noise': 'red', 'No noise': 'blue'}, scale=1.1)

for collection in pp.collections:
    collection.set_zorder(200)
    collection.set_label('_nolegend_')

plt.legend(loc='upper left', bbox_to_anchor=(0, 1))
plt.tight_layout()
plt.grid(True, axis='x')
plt.xticks(np.arange(0, 1 + 0.1, 0.1), fontsize=10.7)
plt.yticks(fontsize=10.7)
plt.xlim(0.5, 1.01)
plt.xlabel('Accuracy', fontsize=13)
plt.ylabel('Number of qubits', fontsize=13)
plt.savefig('qcnn_boxplot.png')
plt.show()